# Data Exploration
***
## Overview
This notebook uses APIs from The World Health Organization (WHO) and The World Bank to obtain datasets used to answer the projects main hypothesis questions: Do countries with higher unemployment rate also have higher suicide rates? For each API, the list of available datasets is examined and an apprpriate one is selected. Both sources provide historical data for at least 190 countires around the world. The data is called using requests and refined from a json file to a dictionary using a simple for loop. The dictionary is then converted to a dataframe using the pandas library. Then null values are dropped from each data frame and the columns at cast as the appropriate types. Lastly the dataframes are merged together so data for both suicide rates and unemployment rates are paired together by coubbtry/year in one dataframe object.

This notebook also contains data exploration for country GDP data, provided by The World Bank, which is used to answer a follow up hopytheses: do countries with higher GDP have lower suicide rates?
### Dependencies

In [ ]:
# Dependencies
import requests
import pandas as pd

## WHO Data Exploration
Main Hypothesis is concered with suicide rates and unemployment data. The first of which WHO says they have data for. WHO has an online database, [The Global Health Observatory](https://www.who.int/data/gho), which includes 1000s of datasets (indicators). 
### WHO's Indicators Access Point
To start, we look at the [indicator](https://ghoapi.azureedge.net/api/Indicator/) access point which list all indicators found on the The Global Health Observatory page.

In [ ]:
# Entry point for WHO's indicators
who_url = 'https://ghoapi.azureedge.net/api/Indicator/'

In [ ]:
# Initialize a variable to loop through indicator list
index = 0

# Read API and print out the name of every indicator with the index value
who_data = requests.get(who_url).json()
for indicator in who_data['value']:
    print( index, indicator['IndicatorName'])
    index += 1

We find there are over 2,000 different indicators. Using a simple `crtl-f` search we narrow down the list to just indicators involving _suicide ratess_. the 665th indicator looks like the appropriate dataset to usd based on it's discription.

In [ ]:
who_data['value'][665]

Note the rates are given as [*age standardized*](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/78#:~:text=The%20age%2Dstandardized%20mortality%20rate,of%20the%20WHO%20standard%20population.) and *per 100,000 people*.
### WHO Suicide Rates 

In [ ]:
# Use the IndicatorCode to create entry point for suicide rates data
sui_url = 'https://ghoapi.azureedge.net/api/MH_12'

In [ ]:
# Read data from API
sui_data = requests.get(sui_url).json()
sui_data

Note that rows of data are organized by country, year and sex. We are concerned with data by country and year but only want values measured for both sexes combined so we will drop rows with the sex specific data.

In [ ]:
# Intialize dictionary
sui_dict = {'country': [], 'year': [], 'suicide rate': [], 'sex': []}

In [ ]:
# Loop through json items to store data
for entry in sui_data['value']:
    sui_dict['country'].append(entry['SpatialDim'])
    sui_dict['year'].append(entry['TimeDim'])
    sui_dict['suicide rate'].append(entry['NumericValue'])
    sui_dict['sex'].append(entry['Dim1'])

In [ ]:
# Make dataframe
sui_df = pd.DataFrame(sui_dict)
sui_df.head()

In [ ]:
# Only want both sex values - loc 'sex' == 'BTSX', then drop sex column it's not needed
sui_df = sui_df.loc[sui_df['sex'] == 'BTSX']
sui_df = sui_df.drop(columns = 'sex')

In [ ]:
# Print data types - year is integer
sui_df.info()

The resulting dataframe contains 3,881 rows of clean/non-null data. This data will have to be paired by country and year with data fro unemployment rate. So not all rows may be matched and thereore used.

## World Bank Data Exploration
The second set of data required to answer the main hypothesis is country unemployment rates. [The world bank's api](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation) provides global data on 1,000s of economic and financial indicators.
### World Bank's Indicator Acces Point
A similair [indicators](https://api.worldbank.org/v2/indicator?format=json&per_page=21000) access point is used to view all indicators provided by The World Bank's API.

In [ ]:
# World bank url
wb_url = 'https://api.worldbank.org/v2/indicator?format=json&per_page=21000'

# Initialize variable to store index count
count = 0

# Print out name of every "indicator" in world bank api to find ones of interest
wb_data = requests.get(wb_url).json()
for indicator in wb_data[1]:
    print(count, indicator['name'])
    count += 1

Seraching for *Unemployemnt Rate* we find indicator 9,746 has relevent data.

In [ ]:
# Same thing for unemployment % ...
wb_data[1][9746]

Note *working age* is defined as 15-64 years of age. 
### World Bank Unemployment Rates

In [ ]:
# Url for unemployment rate indicator
uem_url = 'https://api.worldbank.org/v2/country/indicator/JI.UEM.1564.ZS?format=json&per_page=10000'

# Load data as json
uem_data = requests.get(uem_url).json()
uem_data

In [ ]:
# Intiate a dictionary that stores lists for country, year and unemployment rate
uem_dict = {'country': [], 'year': [], 'unemployment rate': []}

# Loop over entries in umeployment json data
for entry in uem_data[1]:
    
    # Append each list in the dictionary with values from current entry
    uem_dict['country'].append(entry['country']['id'])
    uem_dict['year'].append(entry['date'])
    uem_dict['unemployment rate'].append(entry['value'])

# Load the dictionary as a dataframe
uem_df = pd.DataFrame(uem_dict)
uem_df.head()

In [ ]:
# Check for missing data and how data is stored
uem_df.info()

Note that year is stored as a string (object). We will recast this as an integer to make merging with the suicide rates dataframe possible. Also, Unemployment rate only has valid data for 1,413 of the total 8,262 entries. This is most likely that the JOIN database did not have data for certain countriy, year pairs. We drop this data and assume that the missing data does not result in a systematic error (i.e. lack of reporting is not correlated with country unemployment rate.) 

In [ ]:
uem_df['year'] = uem_df['year'].astype(int)

In [ ]:
uem_df.dropna(inplace=True)
uem_df.info()

### World Bank GDP per Capita
The same steps are repeated to obtain GDP per Capita data. The Indicator was found with a simliar search.

In [ ]:
# Use gdp/capita id code by country to get url
gdp_url = 'https://api.worldbank.org/v2/country/indicator/NY.GDP.PCAP.CD?format=json&per_page=20000'
gdp_data = requests.get(gdp_url).json()
gdp_data

In [ ]:
# Intialize dictionary to store columns of gdp_df
gdp_dict = {'country': [], 'year': [], 'gdp per capita': []}

# Loop through json and store values to dict
for entry in gdp_data[1]:
    gdp_dict['country'].append(entry['countryiso3code'])
    gdp_dict['year'].append(entry['date'])
    gdp_dict['gdp per capita'].append(entry['value'])

In [ ]:
# Convert dict to df
gdp_df = pd.DataFrame(gdp_dict)
gdp_df.head()

In [ ]:
# Number of non-null values and data types
gdp_df.info()

Note that the year column is casted as string (object) and GDP/capita only has data for 13,115 of the total 16,492 possible. To make merging with the suicide rates dataframe possible, year is recasted as integer. The null values are also dropped resluting in less data for certain countriess.

In [ ]:
# 'year' column is object type -> int to match other datasets
gdp_df['year'] = gdp_df['year'].astype(int)

In [ ]:
gdp_df = gdp_df.dropna()
gdp_df.info()

## Merge DataFrames
The Suicide rates dataframe has 3,881 rows of data but the unemployment rates datframe only has 1,413 rows of clean data. To make analysis easier, we merge both dataframes together and only keep rows where we have data for both suicide rates and unemployment rates. The result is a dataframe that stores countries's unemployment rates and suicide rates by country and year, where data is available. 

In [ ]:
# Merge suicide rates and unemployment on country and year columns and only keep rows that appear in both of the original dfs.
sui_vs_uem = pd.merge(sui_df, uem_df, on=['country','year'], how='inner')
sui_vs_uem

A total of 1,100 rows of data.

In [ ]:
# Drop Na and check for amount of data
sui_vs_uem = sui_vs_uem.dropna()
sui_vs_uem['country'].value_counts()

In [ ]:
# How many counties only have 1 years worth of data
(sui_vs_uem['country'].value_counts() == 1).sum()

Note that there are 24 countries where data is only avaible for 1 year.

The same thing is repeated for GDP and suicide rates.

In [ ]:
# Merge suicide rates and gdp on country and year columns, only keeping inclusive (inner) rows
sui_vs_gdp = pd.merge(sui_df, gdp_df, on=['country','year'], how='inner')
sui_vs_gdp

In [ ]:
# See how many years of data for each country - 182 countries with at least 7 years of data
sui_vs_gdp['country'].value_counts()

Note 182 total countires, 180 of which have data fro 12 years or more. 

## Save Cleaned up data as CSV files
The two cleaned up dataframes are saved to .csv files so they can be used in the main_analysis.ipynb

In [ ]:
sui_vs_uem.to_csv('Clean_Data/suicide_vs_unemployment_clean', index=False)
sui_vs_gdp.to_csv('Clean_Data/suicide_vs_gdp_clean', index=False)